In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import time

In [26]:
game_df = pd.read_csv("2024-03-06 Listing Data.csv")
seat_coords = pd.read_csv("2024-03-06 Seat Coordinates.csv")

In [27]:
merged_df = pd.merge(game_df, seat_coords[['SECTION_ROW_SEAT_NAME', 'SEAT_CENTER_X', 'SEAT_CENTER_Y']], on=['SECTION_ROW_SEAT_NAME'], how='left')

In [7]:
#df = pd.read_html("https://www.basketball-reference.com/boxscores/?month=10&day=28&year=2023")

In [28]:
#eastern_conference = df[-2]
#western_conference = df[-1]
#westerm conference

In [29]:
team_name_mapping = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BKN',
    'Charlotte Hornets': 'CHA',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHX',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS'
}

In [30]:
def get_full_team_name(abbreviation, team_name_mapping):
    """
    Function to reverse data mapping from above dictionary to easily convert names from basketball reference to 3-Letter
    abbreviations needed to call functions and merge dataframe
    
    Inputs: 
    abbreviation: targeted 3 letter abbreviation you're trying to convery
    team_name_mapping: dictionary of all the teams 
    
    Returns: 
    full_team_name: the full team name for a specified abbreviation
    """
    # Reverse the team_name_mapping dictionary
    reverse_mapping = {v: k for k, v in team_name_mapping.items()}
    
    # Get the full team name
    full_team_name = reverse_mapping.get(abbreviation, abbreviation)  # If not found, return the abbreviation
    
    return full_team_name

In [31]:
def get_win_percentage(date, team1):
    
    """
    Scrapes and retrieves the win percentage of a given team for a specific date from basketball-reference.com.

    Inputs:
    - date: Date for which the win percentage is requested (in the format 'YYYY-MM-DD').
    - team1: The abbreviation or full name of the team for which win percentage is to be retrieved.

    Returns:
    A DataFrame containing the win percentage of the specified team for the given date.

    Note:
    - This function scrapes data from basketball-reference.com for the specified date.
    - The team name mapping must be provided to map team names to their abbreviations.
    """
    
    result_df = pd.DataFrame(columns=['Date', 'Team', 'Win_Percentage'])

    # Define the start and end dates for scraping
    date = pd.Timestamp(date)

    # Generate the URL for the current date
    url = f"https://www.basketball-reference.com/boxscores/?month={date.month}&day={date.day}&year={date.year}"

    east_df = pd.read_html(url)[-2]
    east_df = east_df.rename(columns={'Eastern Conference': 'Team'})
    
    west_df = pd.read_html(url)[-1]
    west_df = west_df.rename(columns={'Western Conference': 'Team'})

    combined_df = pd.concat([east_df, west_df])
    combined_df = combined_df[['Team', 'W/L%']]
    combined_df.loc[:, 'Team'] = combined_df.loc[:, 'Team'].str.replace('*', '', regex = True)
    teams_df = combined_df[combined_df.loc[:, 'Team'].isin([team1])]
    teams_df.loc[:, 'Date'] = date    
    teams_df.loc[:, 'Team'] = teams_df.loc[:, 'Team'].map(team_name_mapping)


    return teams_df

In [32]:
get_win_percentage('2022-12-30', 'Los Angeles Lakers')

C:\Users\Darin\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\Darin\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,Team,W/L%,Date
12,LAL,0.417,2022-12-30


In [33]:
# create a df to find each individual dates
aggregated_df = merged_df.groupby('GAME_DATE').agg({
    'GAME_ID': 'first',
    'GAME_TIME': 'first',
    'SEASON_ID': 'first',
    'SEASON_STAGE': 'first',
    'VISITOR_TEAM_TRI_CODE': 'first',
    'HOME_TEAM_TRI_CODE': 'first',
    'SECTION_CATEGORY': 'first',
    'SECTION_GROUP': 'first',
    'SECTION_LEVEL': 'first',
    'SECTION_NAME': 'first',
    'ROW_NAME': 'first',
    'SEAT_NAME': 'first',
    'SECTION_ROW_SEAT_NAME': 'first',
    'TICKET_LISTING_PRICE': 'first',
    'DAYS_INVENTORY_LEAD_TIME': 'first',
    'INVENTORY_LEAD_TIME_CATEGORY': 'first',
    'SEAT_CENTER_X': 'first',
    'SEAT_CENTER_Y': 'first'
}).reset_index()

aggregated_df

,GAME_DATE,GAME_ID,GAME_TIME,SEASON_ID,SEASON_STAGE,VISITOR_TEAM_TRI_CODE,HOME_TEAM_TRI_CODE,SECTION_CATEGORY,SECTION_GROUP,SECTION_LEVEL,SECTION_NAME,ROW_NAME,SEAT_NAME,SECTION_ROW_SEAT_NAME,TICKET_LISTING_PRICE,DAYS_INVENTORY_LEAD_TIME,INVENTORY_LEAD_TIME_CATEGORY,SEAT_CENTER_X,SEAT_CENTER_Y
0,2022-10-18,22200002,19:00:00,2022,REGULAR SEASON,LAL,GSW,PREMIUM,LOWER BOWL,LEVEL 100,SECTION 104,ROW 16,SEAT 11,SECTION 104 - ROW 16 - SEAT 11,613.0,0.0,EVENT DAY,2674.0,3604.0
1,2022-10-21,22200026,19:00:00,2022,REGULAR SEASON,DEN,GSW,PREMIUM,LOWER BOWL,LEVEL 100,SECTION 126,ROW 15,SEAT 17,SECTION 126 - ROW 15 - SEAT 17,51.0,0.0,EVENT DAY,439.0,1740.0
2,2022-10-23,22200042,17:30:00,2022,REGULAR SEASON,SAC,GSW,PREMIUM,LOWER BOWL,LEVEL 100,SECTION 115,ROW 07,SEAT 13,SECTION 115 - ROW 07 - SEAT 13,206.0,0.0,EVENT DAY,2706.0,1612.0
3,2022-10-27,22200068,19:00:00,2022,REGULAR SEASON,MIA,GSW,PREMIUM,LOWER BOWL,LEVEL 100,SECTION 114,ROW 15,SEAT 12,SECTION 114 - ROW 15 - SEAT 12,119.0,0.0,EVENT DAY,3042.0,1441.0
4,2022-11-07,22200156,19:00:00,2022,REGULAR SEASON,SAC,GSW,NON-PREMIUM,UPPER BOWL,LEVEL 200,SECTION 218,ROW 08,SEAT 17,SECTION 218 - ROW 08 - SEAT 17,76.0,0.0,EVENT DAY,3250.0,884.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2024-03-22,22301016,19:00:00,2023,REGULAR SEASON,IND,GSW,NON-PREMIUM,UPPER BOWL,LEVEL 200,SECTION 217,ROW 05,SEAT 2,SECTION 217 - ROW 05 - SEAT 2,160.0,0.0,EVENT DAY,3298.0,3287.0
78,2024-04-02,22300589,19:00:00,2023,REGULAR SEASON,DAL,GSW,NON-PREMIUM,UPPER BOWL,LEVEL 200,SECTION 204,ROW 12,SEAT 5,SECTION 204 - ROW 12 - SEAT 5,30.0,0.0,EVENT DAY,1858.0,4224.0
79,2024-04-07,22301142,17:30:00,2023,REGULAR SEASON,UTA,GSW,NON-PREMIUM,UPPER BOWL,LEVEL 200,SECTION 207,ROW 18,SEAT 21,SECTION 207 - ROW 18 - SEAT 21,89.0,13.0,>10 DAYS,2898.0,4380.0
80,2024-04-12,22301182,19:00:00,2023,REGULAR SEASON,NOP,GSW,PREMIUM,LOWER BOWL,LEVEL 100,SECTION 101,ROW 06,SEAT 5,SECTION 101 - ROW 06 - SEAT 5,400.0,18.0,>15 DAYS,1782.0,3319.0


In [15]:
def pull_win_percentages(game_df, team_name_mapping):
    """
    Pulls win percentages for each game in the provided DataFrame, incorporating team name mapping.

    Inputs:
    - game_df: DataFrame containing game data with columns 'GAME_DATE', 'VISITOR_TEAM_TRI_CODE', and 'HOME_TEAM_TRI_CODE'.
    - team_name_mapping: Dictionary mapping team abbreviations to full team names.

    Returns:
    A DataFrame containing win percentages for each game, with columns 'Date', 'Home_Team', 'Home_Win_Percentage',
    'Visitor_Team', and 'Visitor_Win_Percentage'.

    Note:
    - This function iterates over each row in the input DataFrame and retrieves win percentages for both the home and
      visitor teams using the 'get_win_percentage' function.
    - It incorporates a sleep time of 5 seconds between each call to 'get_win_percentage' to avoid overloading the server
      with too many requests.
    """
    
    all_win_percentages = pd.DataFrame()  # Create an empty DataFrame to store win percentages

    
    for index, row in game_df.iterrows():
        date = row['GAME_DATE']
        visitor_team = row['VISITOR_TEAM_TRI_CODE']
        home_team = row['HOME_TEAM_TRI_CODE']
            
        home_team_full = get_full_team_name(home_team, team_name_mapping)
        visitor_team_full = get_full_team_name(visitor_team, team_name_mapping)

        time.sleep(5)
        visitor_win_percentage_df = get_win_percentage(date, visitor_team_full)
        time.sleep(5)
        home_win_percentage_df = get_win_percentage(date, home_team_full)
        time.sleep(5)
        
        print("Date: ", date)
        
        all_win_percentages = all_win_percentages.append({
            'Date': date,
            'Home_Team': home_team,
            'Home_Win_Percentage': home_win_percentage_df["W/L%"].iloc[0],
            'Visitor_Team': visitor_team,
            'Visitor_Win_Percentage': visitor_win_percentage_df["W/L%"].iloc[0]
        }, ignore_index=True)
        
        
    return all_win_percentages

In [34]:
# I RAN THIS FUNCTION ONCE. BECAUSE OF SLEEP TIME IT TAKES FOREVER, SO I JUST SAVED THE OUTPUT AS A CSV FOR CONSISTENT USAGE
#date_team_win_percent_df = pull_win_percentages(aggregated_df, team_name_mapping)

In [35]:
date_team_win_percent_df = pd.read_csv("win_percentages.csv")
date_team_win_percent_df

,Date,Home_Team,Home_Win_Percentage,Visitor_Team,Visitor_Win_Percentage
0,2022-10-18,GSW,1.000,LAL,0.000
1,2022-10-21,GSW,0.500,DEN,0.500
2,2022-10-23,GSW,0.667,SAC,0.000
3,2022-10-27,GSW,0.600,MIA,0.333
4,2022-11-07,GSW,0.364,SAC,0.333
...,...,...,...,...,...
77,2024-03-22,GSW,0.522,IND,0.563
78,2024-04-02,GSW,0.547,DAL,0.600
79,2024-04-07,GSW,0.551,UTA,0.372
80,2024-04-12,GSW,0.556,NOP,0.605


In [38]:
# Merge the DataFrames based on matching dates and visitor team tri codes
final_df = pd.merge(merged_df, date_team_win_percent_df, how='left', left_on=['GAME_DATE', 'VISITOR_TEAM_TRI_CODE'], right_on=['Date', 'Visitor_Team'])

In [39]:
final_df = final_df.drop(["Home_Team", "Visitor_Team", "Date"], axis=1)

In [40]:
final_df

,GAME_ID,GAME_DATE,GAME_TIME,SEASON_ID,SEASON_STAGE,VISITOR_TEAM_TRI_CODE,HOME_TEAM_TRI_CODE,SECTION_CATEGORY,SECTION_GROUP,SECTION_LEVEL,...,ROW_NAME,SEAT_NAME,SECTION_ROW_SEAT_NAME,TICKET_LISTING_PRICE,DAYS_INVENTORY_LEAD_TIME,INVENTORY_LEAD_TIME_CATEGORY,SEAT_CENTER_X,SEAT_CENTER_Y,Home_Win_Percentage,Visitor_Win_Percentage
0,22200002,2022-10-18,19:00:00,2022,REGULAR SEASON,LAL,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 16,SEAT 11,SECTION 104 - ROW 16 - SEAT 11,613.0,0.0,EVENT DAY,2674.0,3604.0,1.000,0.000
1,22200002,2022-10-18,19:00:00,2022,REGULAR SEASON,LAL,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 26,SEAT 2,SECTION 124 - ROW 26 - SEAT 2,480.0,0.0,EVENT DAY,1366.0,3577.0,1.000,0.000
2,22200002,2022-10-18,19:00:00,2022,REGULAR SEASON,LAL,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 20,SEAT 3,SECTION 124 - ROW 20 - SEAT 3,470.0,0.0,EVENT DAY,1446.0,3436.0,1.000,0.000
3,22200002,2022-10-18,19:00:00,2022,REGULAR SEASON,LAL,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 11,SEAT 8,SECTION 114 - ROW 11 - SEAT 8,595.0,0.0,EVENT DAY,2962.0,1522.0,1.000,0.000
4,22200002,2022-10-18,19:00:00,2022,REGULAR SEASON,LAL,GSW,NON-PREMIUM,UPPER BOWL,LEVEL 200,...,ROW 21,SEAT 8,SECTION 221 - ROW 21 - SEAT 8,300.0,0.0,EVENT DAY,2146.0,548.0,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459301,22301198,2024-04-14,12:30:00,2023,REGULAR SEASON,UTA,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 10,SEAT 3,SECTION 110 - ROW 10 - SEAT 3,228.0,119.0,>90 DAYS,3623.0,2124.0,0.561,0.378
459302,22301198,2024-04-14,12:30:00,2023,REGULAR SEASON,UTA,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 20,SEAT 11,SECTION 121 - ROW 20 - SEAT 11,200.0,120.0,>90 DAYS,1147.0,2196.0,0.561,0.378
459303,22301198,2024-04-14,12:30:00,2023,REGULAR SEASON,UTA,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 20,SEAT 12,SECTION 121 - ROW 20 - SEAT 12,200.0,120.0,>90 DAYS,1142.0,2220.0,0.561,0.378
459304,22301198,2024-04-14,12:30:00,2023,REGULAR SEASON,UTA,GSW,PREMIUM,LOWER BOWL,LEVEL 100,...,ROW 20,SEAT 9,SECTION 121 - ROW 20 - SEAT 9,200.0,120.0,>90 DAYS,1157.0,2148.0,0.561,0.378
